# Introduction

- This script is used to get the local electricity mixes (based on garantee of origin) given the data published on [Stromkennzeichnung](https://www.strom.ch/de/service/stromkennzeichnung) in Aug 2019
- Since the website above was restructured, the script below may not work with current version of the website, thus a pkl file saved in Aug 2019 is provided, while the script shows how the data was originally obatined using [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).
- The data contains the % electricity mix by technology for 487 utility providers/entities in 26 Cantons in Switzerland in 2018. The percentages for each utility providers/entities can be added up to be 100%.

For any questions, please contact Xiaojin Zhang xiaojin.zhang@psi.ch

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
all_kantons=["ZH","BE","LU", "UR", "SZ", "OW", "NW", "GL", "ZG", "FR", "SO", "BS", "BL", "SH", "AR", "AI", "SG", "GR", "AG", "TG", "TI", "VD", "VS", "NE", "GE", "JU"]

In [3]:
len(all_kantons)

26

In [4]:
results_2018={}

In [19]:
for kanton in all_kantons:
    url='https://www.stromkennzeichnung.ch/de/suche/pcanton/'+kanton+'.html'
    data_per_kanton=requests.get(url)
    data_per_kanton_soup=BeautifulSoup(data_per_kanton.text, 'html.parser')
    data_per_kanton=data_per_kanton_soup.find_all('div', attrs={'class':'row hoverEffect'})
    records = {}
    for each_supplier in data_per_kanton:
        supplier = each_supplier.find('a').text[4:-3]
        if each_supplier.find('i') is not None:
            elec_type = 'N.A'
            pct = 'N.A'
        else:
            supplies={}
            for elec_supply in each_supplier.find_all('div', attrs={'class':'bar'}):
                elec_type = elec_supply["onmouseover"][13:].split(",")[0].split("_")[0]
                pct = elec_supply["onmouseover"][13:].split(",")[1].split("%")[0][2:]
                supplies.update({elec_type: pct})
            records[supplier]=supplies
    results_2018[kanton]=records

In [23]:
results_2018=pd.DataFrame.from_dict({(i,j): results_2018[i][j] 
                           for i in results_2018.keys() 
                           for j in results_2018[i].keys()},
                       orient='index')

In [94]:
results_2018

Wasserkraft  \
Kanton Supplier                                                          
AG     AEW Energie AG                                            66.92   
       EFA Energie Freiamt AG                                    71.70   
       ELEKTRA Rudolfstetten-Friedlisberg AG (ERF)               93.40   
       EV Gebenstorf AG                                          33.80   
       EV Zeiningen                                              90.57   
       EW Oftringen AG                                           71.09   
       EW Rothrist AG                                            37.30   
       EW Staufen                                                93.90   
       EWK Energie AG                                            76.10   
       EWS Energie AG                                            77.30   
       Einwohnergemeinde Oberwil-Lieli c/o Elektra Obe...         1.40   
       Elektra - Genossenschaft Baldingen                        57.57   
       Elektra Besenbüren                                        90.60   
       Elektra Böttstein                                          1.96   
       Elektra Eggenwil                                            NaN   
       Elektra Geltwil                                           91.51   
       Elektra Genossenschaft Abtwil                             89.05   
       Elektra Genossenschaft Auw                                80.54   
       Elektra Genossenschaft Kaiserstuhl   c/o Elektr...        94.40   
       Elektra Genossenschaft Leibstadt                          17.50   
       Elektra Genossenschaft Oberlunkhofen                      94.15   
       Elektra Genossenschaft Rietheim   c/o Elektrizi...        37.20   
       Elektra Genossenschaft Schneisingen                       93.70   
       Elektra Mettauertal und Umgebung                          47.82   
       Elektra Remetschwil Genossenschaft                        93.60   
       Elektra Sins                                              43.29   
       Elektra Sisseln                                           66.90   
       Elektra Unterlunkhofen                                      NaN   
       Elektra Widen Betriebs AG                                   NaN   
       Elektra Zufikon                                           22.80   
...                                                                ...   
ZG     Ägerital Energie Genossenschaft                             NaN   
ZH     EGW Weiach Elektrizitätsgenossenschaft                    94.38   
       EW Wald AG                                                57.92   
       Elektrizitätsgenossenschaft Bubikon                       93.50   
       Elektrizitätsgenossenschaft Otelfingen                    94.10   
       Elektrizitätsgenossenschaft Rümikon EGR (ZH)              90.40   
       Elektrizitätsversorgung Benken (ZH)                       93.70   
       Elektrizitätswerk Andelfingen c/o Gemeinde Ande...        94.21   
       Elektrizitätswerk Fehraltorf                              65.10   
       Elektrizitätswerk Fällanden                               94.03   
       Elektrizitätswerk Herrliberg                              93.80   
       Elektrizitätswerk Rümlang Genossenschaft                  93.90   
       Elektrizitätswerk der Gemeinde Lindau                     94.27   
       Elektrizitätswerke des Kantons Zürich (EKZ)               82.62   
       Energie Uetikon AG c/o Energie und Wasser Meile...        70.69   
       Energie Uster AG                                          92.12   
       Gemeindewerke Horgen                                      88.40   
       Gemeindewerke Pfäffikon                                   94.24   
       Gemeindewerke Stäfa                                       71.60   
       Genossenschaft Elektrizitätswerk Räterschen EWR           92.63   
       Genossenschaft Licht- und Kraftwerke Glattfelden          94.04   
       Industrielle Betriebe Kloten AG                         

In [47]:
results_2018.index.names="Kanton","Supplier"

In [82]:
cols=['Wasserkraft', 
      'Kernenergie', 
      'Nicht überprüfbare Energieträger', 
      'Geförderter Strom', 
      'Erdgas', 'Erdöl', 'Kohle', 
      'Sonnenenergie', 'Windenergie','Abfälle', 'Biomasse'#, 'Geothermie'
     ]

In [84]:
results_2018=results_2018[cols]

In [74]:
len(results_2018)

487

In [78]:
for column in results_2018.columns:
    results_2018[column] = results_2018[column].astype(float)

In [116]:
results_2018=results_2018.unstack(0)

In [117]:
results_2018.index = results_2018.index.map(lambda x: x.rstrip())

In [123]:
results_2018=results_2018.stack(1)

In [129]:
results_2018=pd.DataFrame.swaplevel(results_2018).sort_index()

In [80]:
results_2018.to_pickle("results_2018_stromkennzeichnung.pkl")

In [112]:
results_2018 = pd.read_pickle("results_2018_stromkennzeichnung.pkl")